In [4]:
from daily_data import daily_data
print(len(daily_data))

720


In [5]:
from collections import namedtuple
Day = namedtuple('Day', ['total_payout', 'total_shares', 'unclaimed_sats', 'payout_acc'])

In [6]:
def decode_raw_day(day):
    bytes_day = day.to_bytes(32, 'big')
    total_payout = int.from_bytes(bytes_day[-9:], 'big')
    total_shares = int.from_bytes(bytes_day[-18:-9], 'big')
    unclaimed_sats = int.from_bytes(bytes_day[:-18], 'big')
    return total_payout, total_shares, unclaimed_sats


In [19]:
SCALE = 10**18

def require_is_uint256(x):
    if type(x) is not int:
        raise TypeError('x not of type int')
    if x >= (1 << 256):
        raise ValueError('x exceeds uint256 size')
    return x
        
SCALE

1000000000000000000

In [43]:
full_daily_data = []
next_acc = 0
for raw_day in daily_data[1:]:
    total_payout, total_shares, unclaimed_sats = decode_raw_day(raw_day)
    full_daily_data.append(Day(total_payout, total_shares, unclaimed_sats, next_acc))
    if total_shares > 0:
        next_acc += require_is_uint256(total_payout * SCALE // total_shares)
    
full_daily_data.append(Day(None, None, None, next_acc))
total_days = len(full_daily_data) - 1
full_daily_data[:3]

[Day(total_payout=55329220332766, total_shares=501364909947909436, unclaimed_sats=1795387317713001, payout_acc=0),
 Day(total_payout=71541552620633, total_shares=761531175093469456, unclaimed_sats=1794410398133809, payout_acc=110357185425111),
 Day(total_payout=97938185332763, total_shares=1013794810629683382, unclaimed_sats=1794047382764103, payout_acc=204301537226404)]

In [44]:
from random import random, randint, seed
import math

In [45]:
math.log(100_000 / 1, 10)

5.0

In [46]:
def randrange(low, high=None):
    if high is None:
        high = low
        low = 0
    return (high - low) * random() + low

In [47]:
def lin_random_bn(low, high, s=1e12):
    return randint(int(low * s), int(high * s))

def log_random_bn(low, high, s=1e12):
    log_diff = math.log(high / low)
    return int(low * math.exp(randrange(log_diff)) * s)

def rand_sign(neg_prob):
    if random() <= neg_prob:
        return -1
    return 1
        

In [48]:
def full_calculate_payout(user_shares, start_day, end_day, extra_scale=1):
    return sum(
        user_shares * total_payout * extra_scale // total_shares
        for total_payout, total_shares, _, _ in full_daily_data[start_day:end_day]
    ) // extra_scale

In [49]:
def acc_calculate_payout(user_shares, start_day, end_day):
    start_acc = full_daily_data[start_day].payout_acc
    end_acc = full_daily_data[end_day].payout_acc
    return user_shares * (end_acc - start_acc) // SCALE
    

In [50]:
def calculate_errors(user_shares, start_day, end_day, extra_precision):
    full_payout = full_calculate_payout(user_shares, start_day, end_day, extra_precision)

    acc_payout = acc_calculate_payout(user_shares, start_day, end_day)

    acc_error = acc_payout / full_payout - 1
    abs_acc_error = acc_payout - full_payout
    print(f'\nerror at x{float(extra_precision)} added precision:')
    print(f'calculation error: {acc_error} (+{abs_acc_error:,} Hearts)')

    relative_error = abs_acc_error / span
    print(f'relative error: {relative_error:,.4f} Hearts / Day')
    

In [394]:
# RUN this cell to simulate single stake

user_shares = log_random_bn(0.01, 1e3)
print(f'user shares: {user_shares / 1e12:,.6f} T-Shares')

maximize_range = False

if maximize_range:
    start_day = randint(0, 10)
    end_day = randint(total_days-10, total_days)
else:
    start_day = randint(0, total_days - 1)
    end_day = randint(start_day + 1, total_days)
span = end_day - start_day
print(f'day {start_day} - {end_day} (span: {span})')

full_payout = full_calculate_payout(user_shares, start_day, end_day)
print(f'full payout: {full_payout/1e8:,.2f} HEX')

acc_payout = acc_calculate_payout(user_shares, start_day, end_day)
print(f'accumulated payout: {acc_payout/1e8:,.2f} HEX')

params = user_shares, start_day, end_day
calculate_errors(*params, 10**0)
calculate_errors(*params, 10**1)
calculate_errors(*params, 10**2)
calculate_errors(*params, 10**3)
calculate_errors(*params, 10**4)


user shares: 0.016374 T-Shares
day 461 - 468 (span: 7)
full payout: 0.67 HEX
accumulated payout: 0.67 HEX

error at x1.0 added precision:
calculation error: 4.4647605612624375e-08 (+3 Hearts)
relative error: 0.4286 Hearts / Day

error at x10.0 added precision:
calculation error: 1.4882534760118915e-08 (+1 Hearts)
relative error: 0.1429 Hearts / Day

error at x100.0 added precision:
calculation error: 0.0 (+0 Hearts)
relative error: 0.0000 Hearts / Day

error at x1000.0 added precision:
calculation error: 0.0 (+0 Hearts)
relative error: 0.0000 Hearts / Day

error at x10000.0 added precision:
calculation error: 0.0 (+0 Hearts)
relative error: 0.0000 Hearts / Day
